In [292]:
# 加载模型
import torch
from stable_baselines3 import DQN
import pandas as pd

# model = DQN.load("./model/SACE_dqn_model",device='cpu')
# model_lane_change = model.policy.q_net.q_net
model_path = "dqn_follow.zip"
device = 'cpu'  # 如果需要在 GPU 上推断，可以改为 'cuda'
model = DQN.load(model_path, device=device)
model_q_net = model.policy.q_net.q_net
# data = torch.load("./data/SACE_dqn_data.pth")
# data_x = torch.stack(data)

Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)


In [293]:
data_pd = pd.read_csv("data_1.csv")
data_pd['field.f1_speed'] = data_pd['field.f1_speed'] - data_pd['field.ego_speed']  
data_x = torch.tensor(data_pd.values, dtype=torch.float32)


In [294]:
# import pandas as pd
# import shap
# import matplotlib.font_manager as fm
# import matplotlib
# matplotlib.use('TkAgg')  # 或其他支持动画的后端，如 'Qt5Agg'
# import pickle
# import pandas as pd
# import shap
# # 设置 Matplotlib 的默认样式
# import matplotlib.pyplot as plt
# plt.style.use('default')
# plt.rcParams['savefig.format'] = 'svg'
#
# # 设置 Matplotlib 的默认背景色为白色
# plt.rcParams['axes.facecolor'] = 'white'

In [295]:
from FullyNet_Shapleyflowtest import FullyNet_Shapleyflow

In [296]:
model_q_net(data_x)

tensor([[ 1.5900e+00,  2.0237e+00,  2.7224e+00],
        [ 1.2965e+00,  1.8472e+00,  2.5823e+00],
        [ 1.2738e+00,  1.8281e+00,  2.5648e+00],
        [ 1.2738e+00,  1.8281e+00,  2.5648e+00],
        [ 7.8385e-01,  1.5114e+00,  2.2202e+00],
        [ 7.8385e-01,  1.5114e+00,  2.2202e+00],
        [ 7.8385e-01,  1.5114e+00,  2.2202e+00],
        [ 7.8385e-01,  1.5114e+00,  2.2202e+00],
        [ 4.7933e-01,  1.2923e+00,  1.9942e+00],
        [ 4.7933e-01,  1.2923e+00,  1.9942e+00],
        [ 4.7933e-01,  1.2923e+00,  1.9942e+00],
        [ 4.7933e-01,  1.2923e+00,  1.9942e+00],
        [ 3.5724e-01,  1.1728e+00,  1.8876e+00],
        [ 3.5724e-01,  1.1728e+00,  1.8876e+00],
        [ 3.5724e-01,  1.1728e+00,  1.8876e+00],
        [ 3.5724e-01,  1.1728e+00,  1.8876e+00],
        [ 4.3171e-01,  1.2347e+00,  1.9471e+00],
        [ 4.3171e-01,  1.2347e+00,  1.9471e+00],
        [ 4.3171e-01,  1.2347e+00,  1.9471e+00],
        [ 4.3171e-01,  1.2347e+00,  1.9471e+00],
        [ 4.5217e-01

In [297]:
# max_indices = torch.argmax(model_q_net(data_x), dim=1)
# 找到每组输出的最大值及其索引
max_values, max_indices = torch.max(model_q_net(data_x), dim=1) # 所有被选择的决策索引，及其输出值
# 找到最大值中的最小值的索引
min_indices_in_max_values = torch.argmin(max_values)  # 输出值最小的索引，正定基值点索引

In [298]:
background = data_x[min_indices_in_max_values:min_indices_in_max_values+1]

In [299]:
test_aa = FullyNet_Shapleyflow(model_q_net, background)
aaaaa = test_aa.calculate_shapley_flow(data_x)

## 绘图

### 局部解释图

#### 局部减速解释

In [300]:
# 找到元素是 0 的索引
indices_0 = torch.nonzero(max_indices == 0).squeeze()

# 找到元素是 1 的索引
indices_1 = torch.nonzero(max_indices == 1).squeeze()

# 找到元素是 2 的索引
indices_2 = torch.nonzero(max_indices == 2).squeeze()

In [301]:
import shap
sample_index = indices_0[0]   # 样本索引
out_index = 0      # 输出索引
shapley_value = aaaaa[sample_index][out_index].numpy()
base_value = model_q_net(background)[0][out_index].detach().numpy()

In [302]:
model_q_net(background)

tensor([[-0.5662, -0.6229, -0.7714]], grad_fn=<AddmmBackward0>)

In [303]:
expl = shap.Explanation(values=shapley_value, base_values=base_value.item())
expl.data = data_x[sample_index].numpy()
obs_name = ['$\Delta d$', '$\Delta v$', '$v$']
expl.feature_names = obs_name

In [304]:
import matplotlib
matplotlib.use('TkAgg')  # 或其他支持动画的后端，如 'Qt5Agg'
import pickle
import pandas as pd
import shap
# 设置 Matplotlib 的默认样式
import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['savefig.format'] = 'svg'
%matplotlib inline
shap.waterfall_plot(expl, show=False)
plt.savefig('figure/局部减速.svg',dpi=600, bbox_inches='tight')
plt.close()
# plt.cla()

#### 局部保持解释

In [305]:
sample_index = indices_1[0]  # 样本索引
out_index = 1  # 输出索引
shapley_value = aaaaa[sample_index][out_index].numpy()
base_value = model_q_net(background)[0][out_index].detach().numpy()
expl = shap.Explanation(values=shapley_value, base_values=base_value.item())
expl.data = data_x[sample_index].numpy()
expl.feature_names = obs_name


In [306]:
plt.style.use('default')
plt.rcParams['savefig.format'] = 'svg'
%matplotlib inline
shap.waterfall_plot(expl, show=False)
plt.savefig('figure/局部保持.svg', dpi=600, bbox_inches='tight')
plt.close()
# plt.cla()

#### 局部加速解释

In [307]:
sample_index = indices_2[55]  # 样本索引
out_index = 2  # 输出索引
shapley_value = aaaaa[sample_index][out_index].numpy()
base_value = model_q_net(background)[0][out_index].detach().numpy()
expl = shap.Explanation(values=shapley_value, base_values=base_value.item())
expl.data = data_x[sample_index].numpy()
expl.feature_names = obs_name


plt.style.use('default')
plt.rcParams['savefig.format'] = 'svg'
%matplotlib inline
shap.waterfall_plot(expl, show=False)
plt.savefig('figure/局部加速.svg', dpi=600, bbox_inches='tight')
plt.close()

In [308]:
sample_index = indices_1[0]  # 样本索引
out_index = 1  # 输出索引
shapley_value = aaaaa[sample_index][out_index].numpy()
base_value = model_q_net(background)[0][out_index].detach().numpy()

base_value

array(-0.62290776, dtype=float32)

In [309]:
base_value.item()

-0.6229077577590942

In [310]:
model_q_net(background)

tensor([[-0.5662, -0.6229, -0.7714]], grad_fn=<AddmmBackward0>)

### 全局解释图

In [311]:
stacked_tensor = torch.stack(aaaaa)
obs_name = ['$d$', '$\Delta v$', '$v$']
shap.summary_plot(stacked_tensor[indices_0, 0,:].numpy(), data_x[indices_0].numpy(), feature_names=obs_name, show=False)
plt.savefig('figure/减速情况1.svg',dpi=600, bbox_inches='tight')
plt.close()

In [312]:

shap.summary_plot(stacked_tensor[indices_1, 1, :].numpy(), data_x[indices_1].numpy(), feature_names=obs_name, show=False)
plt.savefig('figure/保持情况1.svg',dpi=600, bbox_inches='tight')
plt.close()

In [314]:
stacked_tensor = torch.stack(aaaaa)

shap.summary_plot(stacked_tensor[indices_2, 2, :].numpy(), data_x[indices_2].numpy(), feature_names=obs_name, show=False)
plt.savefig('figure/加速情况1.svg',dpi=600, bbox_inches='tight')
plt.close()